# Si446x Device Direct Access Layer Tests

This notebook explores the Si446xRadio class, which handles the direct access to operating system provided SPI bus and GPIO interface pins that connect to the Si446x device. This provides the command and control interface provided by the device.

In [ ]:
from __future__ import print_function

import sys
sys.path.append("../") # go to parent dir
%run si446x_Device_Layer.ipynb

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))
print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
radio = si446x_device_start_radio()

In [ ]:
si446x_device_show_config(radio.dump_radio())

## Check for Command Error

In [ ]:
status = radio.get_chip_status()
if (status.chip_pend.CMD_ERROR):
    print(status)

##  Configure Radio

In [ ]:
config = si446x_device_config_radio(radio)

si446x_device_show_config(radio.dump_radio())
total = 0
print('\n=== const config strings:')
for s in config:
    print((hexlify(s)))
    total += len(s) - 4
print('\n total: {}'.format(total))

## Set Interrupt Control

In [ ]:
radio.set_property('INT_CTL', 0, '\x03\x3b\x2b\x00')

In [ ]:
s_name = 'int_ctl_group_s'
prop = radio.get_property('INT_CTL', 0, eval(s_name).sizeof())
print('get interrupt property group: {}'.format(insert_space(prop)))
print(si446x_device_format_group(s_name, prop))

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Get Interrupt Status

In [ ]:
s_name =  'int_status_rsp_s'
prop = eval(s_name).build(radio.get_interrupts())
print('{}: {}'.format(s_name, hexlify(prop)))
print(si446x_device_format_group(s_name, prop))

## Set Transmit Power

In [ ]:
s_name =  'pa_group_s'
pa = si446x_device_get_group(radio, 'PA')
print(si446x_device_format_group('PA', eval(s_name).build(pa)))
radio.set_power(120)
pa = si446x_device_get_group(radio, 'PA')
print(si446x_device_format_group('PA', eval(s_name).build(pa)))
print(pa_group_s.parse(eval(s_name).build(pa)))

In [ ]:
pa = si446x_device_get_group(radio, 'INT_CTL')
print(pa)

## Get Properties  by Group

In [ ]:
group_str = radio_config_groups[radio_config_group_ids.build('PA')]
print(hexlify(si446x_device_get_property(radio, 'PA', 0, group_str.sizeof())))
print(si446x_device_get_group(radio, 'PA'))

## Interactive Group Properties

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

## Transmit Packet Stream

In [ ]:
from __future__ import print_function
from time import sleep

num         = 10
delay       = .1
pkt         = bytearray(256)
ss          = bytearray('\x10\x10')

step        = 20
mj_r        = range(num)
th_r        = range(1,45,step)   # fifo threshold setting (min,max): 1, 64
sz_r        = range(4,100,step)  # msg size (min,max): 4, 255
pw_r        = range(10,22,step)  # power amp setting (min, max): 1, 127

radio.trace._enable()

for major in mj_r:
    for thresh in th_r:
        print('thresh: {}'.format(thresh))
        ss[0] = thresh
        ss[1] = thresh
        radio.set_property('PKT', 0x0b, ss) # tx/rx threshold
        for size in sz_r:
            pl = []
            pkt[1] = (size % 256)
            for pw in pw_r:
                pl.append(si446x_device_send_msg(radio, pkt[:size], pw))
                sleep(delay)
            for l in pl:
                if (l[-1] == 'e'):
                    print('ERROR: size:{}, power: {}, results: {}'.format(size, pw, ''.join(map(str, l))))
#           print('{} {}'.format(size, ''.join(map(str, l))))
    print('done {}\n'.format(major))

## Check Interrupt Status

In [ ]:
def int_status(clr_flags=None, show=False):
    s_name =  'int_status_rsp_s'
    p_s = eval(s_name)
    clr_flags = clr_flags if (clr_flags) else clr_pend_int_s.parse('\xff' * clr_pend_int_s.sizeof())
    clr_flags.ph_pend.STATE_CHANGE = False
    p_g = radio.get_clear_interrupts(clr_flags)
    p_d = p_s.build(p_g)
    if (show):
#        print('{}: {}'.format(s_name, hexlify(p_d)))
        print(radio_display_structs[p_s](p_s, p_d))
    return p_g

In [ ]:
def show_int_rsp(pend_flags):
    s_name =  'int_status_rsp_s'
    p_s = eval(s_name)
    clr_flags = clr_pend_int_s.parse('\xff' * clr_pend_int_s.sizeof())
    clr_flags.ph_pend.STATE_CHANGE = False
    p_g = radio.get_clear_interrupts(clr_flags)
    p_d = p_s.build(p_g)
#    print('{}: {}'.format(s_name, hexlify(p_d)))
    print(radio_display_structs[p_s](p_s, p_d))


In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
s_name =  'get_chip_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_chip_status()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
clr_flags = clr_pend_int_s.parse('\xff' * clr_pend_int_s.sizeof())
clr_flags.ph_pend.STATE_CHANGE = False
p_g = radio.get_clear_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
#t,r = si446xradio_test()

In [ ]:
from datetime import datetime 

now=datetime.now()
print(now)